In [1]:
import gym 
import numpy as np
import pandas as pd 
np.random.seed(100)

In [2]:
env = gym.make('CartPole-v0')
env.seed(100)

[100]

In [3]:
action_size = env.action_space.n
action_size

2

In [4]:
[env.action_space.sample() for _ in range(10)]

[1, 1, 0, 1, 1, 0, 0, 1, 0, 0]

In [5]:
state_size = env.observation_space.shape[0]
state_size

4

In [6]:
state = env.reset()
state

array([-0.01628537,  0.02379786, -0.0391981 , -0.01476447], dtype=float32)

In [7]:
state, reward, done, _ = env.step(env.action_space.sample())
state, reward, done, _

(array([-0.01580941,  0.2194594 , -0.03949339, -0.31955263], dtype=float32),
 1.0,
 False,
 {})

In [8]:
%%time 
data = pd.DataFrame()
state = env.reset()
length = []
for run in range(25000):
    done = False
    prev_state = env.reset()
    treward = 1
    results = []
    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
        results.append({'s1': prev_state[0], 's2': prev_state[1],
                        's3': prev_state[2], 's4': prev_state[3], 
                        'a':action, 'r':reward})
        treward += reward if not done else 0
        prev_state = state
    if treward >= 110:
        data = data.append(pd.DataFrame(results))
        length.append(treward)

Wall time: 9.3 s


In [9]:
np.array(length).mean()

119.2

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596 entries, 0 to 120
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   s1      596 non-null    float32
 1   s2      596 non-null    float32
 2   s3      596 non-null    float32
 3   s4      596 non-null    float32
 4   a       596 non-null    int64  
 5   r       596 non-null    float64
dtypes: float32(4), float64(1), int64(1)
memory usage: 23.3 KB


In [11]:
data.tail()

,s1,s2,s3,s4,a,r
116,1.619334,1.116495,-0.172431,0.154536,1,1.0
117,1.641664,1.313612,-0.169340,-0.187199,1,1.0
118,1.667936,1.510701,-0.173084,-0.528153,1,1.0
119,1.698150,1.707782,-0.183647,-0.869989,0,1.0
120,1.732306,1.515569,-0.201047,-0.640208,0,1.0


In [12]:
from pylab import plt 
plt.style.use('seaborn')
%matplotlib inline 

In [13]:
import tensorflow as tf 
tf.random.set_seed(100)

In [14]:
from keras.layers import Dense 
from keras.models import Sequential

In [15]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=env.observation_space.shape[0]))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [16]:
%%time 
history = model.fit(data[['s1','s2','s3','s4']], data['a'],
          epochs=25, verbose=False, validation_split=0.2)

Wall time: 1.67 s


In [17]:
res = pd.DataFrame(history.history)
res.tail(3)

,loss,acc,val_loss,val_acc
22,0.641745,0.617647,0.643975,0.633333
23,0.640972,0.619748,0.640805,0.633333
24,0.640418,0.615546,0.640400,0.633333


In [18]:
def epoch():
    done = False
    state = env.reset()
    treward = 1
    while not done:
        action = np.where(model.predict(np.atleast_2d(state))[0][0] > \
        0.5, 1, 0)
        state, reward, done, _ = env.step(action)
        treward += reward if not done else 0
    return treward


In [19]:
# res = np.array([epoch() for _ in range(100)])
# res.mean()